#### PRIZM Data Prep Pipeline

In [ ]:
from data_prep import DataPrep
from data_binning import *

If getting data from the metadatabase:

In [ ]:
selection = ['70MHz', 'NS', '2021']
calib_type = 'GSM' # 'GSM' for short subtracted data, 'raw' for raw antenna

dataprep = DataPrep(selection[0], selection[1], selection[2])
data, lst, _ = dataprep(calibration_type=calib_type)

If uploading data from saved NumPy files:

In [ ]:
path2file = 'DATA/100/EW/'
file_ending = '_2021_100EW.npy'

with open(path2file + 'data' + file_ending, 'rb') as f:
    data = np.load(f)
with open(path2file + 'lst' + file_ending, 'rb') as f:
    lst = np.load(f)
with open(path2file + 'systime' + file_ending, 'rb') as f:
    systime = np.load(f)
if calib_type == 'GSM':
    with open(path2file + 'short_interp' + file_ending, 'rb') as f:
        short = np.load(f)
    data -= short

Data truncation and RFI excision

In [ ]:
RFI = RFI_flagging(data, lst)
data = RFI.data

LST Binning

In [ ]:
lst_bins, data_binned, _ = lst_binning(data, lst, binsize=2)